In [12]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import sys
import argparse
import overpass
import json
import math
from shapely.geometry import Polygon

%matplotlib inline 

json1_file = open('testFeatureMapping.json')
json1_str = json1_file.read()
mapping = json.loads(json1_str)

In [13]:
# Replace with inline add
observations = gpd.read_file('../data/SidewalkObservations/SidewalkObservations.shp')

In [15]:
# Create a bounding box around the data

bounds = observations.total_bounds
bounds

array([-122.4190186 ,   47.4956156 , -122.23723589,   47.73429618])

In [16]:
xChop = 1
maxX = .05

xChop = math.ceil((bounds[2] - bounds [0])/maxX)
xChop

4

In [17]:
yChop = 1
maxY = .05

yChop = math.ceil((bounds[3] - bounds [1])/maxY)
yChop

5

In [54]:
xSize = ((bounds[2] - bounds [0]) / xChop)
ySize = ((bounds[3] - bounds [1]) / yChop)
api = overpass.API()
MAX_TOL = .003

pointDataFrame = gpd.GeoDataFrame()
ref = -1

for x in range(0, 1):
    for y in range(0, 4):    
        xmin = bounds[0] + (xSize * x)
        xmax = xmin + xSize
        ymin = bounds[1] + (ySize * y)
        ymax = ymin + ySize
        
        polygon = Polygon([(xmin, ymin), (xmin, ymax), (xmax, ymin), (xmax, ymax)])

        currentPoints = observations[observations.geometry.within(polygon)]

        api = overpass.API()
        map_query = overpass.MapQuery(ymin, xmin, ymax, xmax)
        query = 'way [highway=footway] ('+str(ymin)+','+str(xmin)+','+str(ymax)+','+str(xmax)+');<;>;'
        print(query)
        response = api.Get(query)
        print(response)
        OSM = gpd.GeoDataFrame.from_features(response)
        
        
        if(len(OSM.index) > 0):
            
            sidewalks = OSM[OSM.geometry.type == 'LineString']
            
            if(len(sidewalks.index) > 0):
        
                for index, row in currentPoints.iterrows(): 
                    #print(row.geometry)
                    point = row.geometry

                    mylist = [x.object for x in sidewalks.sindex.nearest(point.bounds, 10, objects=True)]
                    distances = sidewalks.loc[mylist].distance(point)

                    if distances.min() < MAX_TOL:
                        nearestIndex = distances.idxmin()
                        sidewalk_geom = sidewalks.loc[nearestIndex].geometry
                        distance_along = sidewalk_geom.project(point)
                        sidewalk_point = sidewalk_geom.interpolate(distance_along)

                        keys = row.keys()

                        mappingDict = {}

                        for key in keys:
                            if key in mapping:
                                if 'categoryMap' in mapping[key]:
                                    value = row[key]
                                    if value in mapping[key]["categoryMap"]:
                                        mappedValue = mapping[key]["categoryMap"][value]
                                        mappingDict[mapping[key]["target"]] = [mappedValue]
                                else:
                                    value = row[key]
                                    if len(value) > 0:
                                        mappingDict[mapping[key]["target"]] = [value]

                        if(len(mappingDict.keys()) > 0):
                            mappingDict['id'] = ref
                            mappingDict['geometry'] = [sidewalk_point]
                            ref = ref - 1

                            pdf = gpd.GeoDataFrame(mappingDict) 

                            pointDataFrame = gpd.GeoDataFrame(pd.concat([pointDataFrame, 
                                            gpd.GeoDataFrame(mappingDict)
                                            ], ignore_index=True))
                    else:
                        print(index)


        



way [highway=footway] (47.495615597,-122.419018602,47.5433517131,-122.373572925);<;>;
{"features": [], "type": "FeatureCollection"}
way [highway=footway] (47.5433517131,-122.419018602,47.5910878292,-122.373572925);<;>;
{"features": [], "type": "FeatureCollection"}
way [highway=footway] (47.5910878292,-122.419018602,47.6388239453,-122.373572925);<;>;
{"features": [], "type": "FeatureCollection"}
way [highway=footway] (47.6388239453,-122.419018602,47.6865600614,-122.373572925);<;>;
{"features": [{"geometry": {"coordinates": [-122.3992983, 47.6646358], "type": "Point"}, "id": 60664566, "properties": {}, "type": "Feature"}, {"geometry": {"coordinates": [-122.3992977, 47.6647245], "type": "Point"}, "id": 60664567, "properties": {}, "type": "Feature"}, {"geometry": {"coordinates": [-122.3989563, 47.664638], "type": "Point"}, "id": 60664568, "properties": {}, "type": "Feature"}, {"geometry": {"coordinates": [-122.398695, 47.6646652], "type": "Point"}, "id": 60664569, "properties": {}, "type":

78056
78059
78061
78064
78066
78067
78073
78076
78077
78078
78082
78083
78089
78091
78093
78094
78095
78097
78098
78099
78100
78101
78103
78106
78108
78109
78111
78113
78117
78118
78120
78126
78131
78135
78136
78137
78139
78140
78142
78144
78146
78147
78149
78151
78152
78154
78155
78156
78158
78159
78162
78163
78164
78166
78167
78170
78171
78172
78174
78176
78177
78180
78182
78183
78185
78188
78192
78193
78198
78199
78201
78202
78203
78205
78206
78209
78212
78213
78216
78217
78218
78220
78224
78229
78233
78239
78244
78246
78248
78251
78253
78254
78255
78257
78259
78260
78264
78265
78266
78268
78270
78272
78273
78274
78280
78283
78284
78286
78288
78289
78292
78294
78296
78304
78306
78310
78311
78313
78315
78320
78321
78326
78330
78332
78333
78334
78338
78340
78341
78342
78345
78348
78350
78354
78355
78359
78363
78368
78369
78370
78374
78375
78377
78379
78380
78384
78394
78405
78414
78418
78421
78431
78434
78438
78440
78442
78449
78454
78457
78460
78477
78495
78509
78519
78521
78532
7853

86695
86699
86701
86712
86719
86720
86725
86728
86733
86734
86738
86741
86743
86748
86751
86753
86757
86758
86759
86766
86769
86774
86779
86784
86787
86791
86798
86801
86809
86812
86814
86820
86824
86825
86830
86831
86835
86836
86842
86846
86849
86858
86861
86864
86865
86867
86869
86871
86874
86876
86878
86882
86883
86888
86889
86895
86900
86904
86909
86910
86916
86926
86930
86931
86934
86937
86938
86939
86943
86944
86945
86949
86950
86957
86959
86967
86970
86972
86977
86979
86980
86985
86987
86992
86997
86998
88358
89354
89360
89372
89376
89381
89388
89393
89396
89404
89410
89416
89420
89426
89433
89442
89446
89561
89569
89580
89581
89584
89588
89594
89596
89600
89605
89611
89617
89623
89642
89648
89651
89655
89656
89657
89672
89673
89676
89678
89688
89689
89697
89698
89700
89703
89708
89714
89717
89733
89743
89758
89760
89767
89770
89773
89776
89793
89800
89801
89808
89816
89821
89827
89867
90018
90031
90042
98228
98233
98235
98245
98249
98371
98375
98381
98395
98408
98422
98438
9844

107946
107947
107948
107950
107951
107952
107953
107954
107955
107956
107957
107958
107959
107960
107961
107963
107964
107965
107966
107967
107969
107970
107971
107972
107973
107974
107975
107976
107977
107978
107979
107980
107981
107982
107983
107984
107986
107988
107992
107993
107994
107995
107996
107997
107998
107999
108330
108340
108484
108504
108518
108526
108541
108549
108553
108563
108568
108573
108582
108588
108595
108597
108603
108608
108618
108621
108626
108647
108663
108666
108669
108680
108684
108688
108692
108700
108704
108705
108709
108714
108720
108730
108737
108745
108752
108763
108767
108775
108793
108801
108805
108809
108811
108816
108817
108839
108846
108852
108854
108859
108860
108861
108869
108919
108925
108927
108933
108935
108938
108942
108969
108974
108983
108991
111001
111008
111014
111020
111026
111032
111052
111057
111063
111084
111085
111098
111105
111108
111114
111119
111120
111130
111139
111147
111154
111157
111159
111162
111164
111166
111171
111175
111179

149527
149528
149529
149530
149531
149532
149533
149534
149535
149536
149537
149538
149539
149540
149541
149542
149543
149544
149545
149546
149547
149548
149557
149558
149559
149560
149561
149562
149563
149564
149565
149802
149803
149804
149805
149806
149807
149808
149809


In [55]:
pointDataFrame

,TestTarget,displacement,geometry,id
0,1,0.42,POINT (-122.3953402 47.6650973),-1
1,1,0.47,POINT (-122.3953402 47.6650973),-2
2,1,0.44,POINT (-122.39964 47.6646356),-3
3,2,1.18,POINT (-122.39964 47.6646356),-4
4,2,1.67,POINT (-122.39964 47.6646356),-5
5,1,0.58,POINT (-122.39964 47.6646356),-6
6,1,0.53,POINT (-122.39964 47.6646356),-7
7,1,0.51,POINT (-122.39964 47.6646356),-8
8,2,0.95,POINT (-122.39964 47.6646356),-9
9,2,0.77,POINT (-122.39964 47.6646356),-10


In [34]:
from lxml import etree
import xml.etree.ElementTree as ET

root = etree.Element("osm")
root.attrib['generator']='CrappySidewalks'

boundsElement = etree.Element('bounds')
boundsElement.attrib['minlat'] = str(bounds[1])
boundsElement.attrib['minlon'] = str(bounds[0])
boundsElement.attrib['maxlat'] = str(bounds[3])
boundsElement.attrib['maxlon'] = str(bounds[2])
boundsElement.attrib['origin'] = "Overpass API"

root.append(boundsElement)


In [158]:
for index, row in pointDataFrame.iterrows(): 
    node = etree.Element("node")
    
    for colName in pointDataFrame.keys():
        if colName == 'id':
            node.attrib['id'] = str(row['id'])
        elif colName == 'geometry':
            node.attrib['lon'] = str(row['geometry'].x)
            node.attrib['lat'] = str(row['geometry'].y)
        else:
            tag = etree.Element("tag")
            tag.attrib['k'] = colName
            tag.attrib['v'] = str(row[colName])
            node.append(tag)
            
    root.append(node)

In [170]:
print(etree.tostring(root[1], pretty_print=True))

b'<node lon="-122.39266866021907" lat="47.522565710960734" id="-1">\n  <tag k="TestTarget" v="1"/>\n  <tag k="displacement" v="0.66"/>\n</node>\n'


In [166]:
root[0]

<Element bounds at 0x2efa3b448>

In [171]:
xmlString = etree.tostring(root, pretty_print=True)

et = etree.ElementTree(root)
et.write('output.xml', pretty_print=True)

In [5]:
api = overpass.API()
way_query = overpass.WayQuery('[name="Highway 51"]')
response = api.Get('(node(50.746,7.154,50.748,7.157);<;>;);')

In [6]:
response

{"features": [{"geometry": {"coordinates": [7.145573, 50.7521218], "type": "Point"}, "id": 26508648, "properties": {}, "type": "Feature"}, {"geometry": {"coordinates": [7.1493367, 50.7521195], "type": "Point"}, "id": 26508649, "properties": {"crossing": "traffic_signals", "highway": "traffic_signals"}, "type": "Feature"}, {"geometry": {"coordinates": [7.150245, 50.7521486], "type": "Point"}, "id": 26508650, "properties": {}, "type": "Feature"}, {"geometry": {"coordinates": [7.15288, 50.752279], "type": "Point"}, "id": 26508652, "properties": {}, "type": "Feature"}, {"geometry": {"coordinates": [7.1555971, 50.752496], "type": "Point"}, "id": 26508653, "properties": {}, "type": "Feature"}, {"geometry": {"coordinates": [7.1572849, 50.7527998], "type": "Point"}, "id": 26508655, "properties": {}, "type": "Feature"}, {"geometry": {"coordinates": [7.1435354, 50.7521656], "type": "Point"}, "id": 26508662, "properties": {"highway": "traffic_signals"}, "type": "Feature"}, {"geometry": {"coordina

In [49]:
response = api.Get('way["highway"="footway"](47.647118264705576,-122.32757091522217,47.673219138520295,-122.28804588317873);>;')



In [50]:
response

{"features": [{"geometry": {"coordinates": [-122.3267809, 47.6543809], "type": "Point"}, "id": 29937535, "properties": {}, "type": "Feature"}, {"geometry": {"coordinates": [-122.3209024, 47.6556194], "type": "Point"}, "id": 29937543, "properties": {}, "type": "Feature"}, {"geometry": {"coordinates": [-122.3155509, 47.6545309], "type": "Point"}, "id": 29937548, "properties": {}, "type": "Feature"}, {"geometry": {"coordinates": [-122.3146043, 47.6542444], "type": "Point"}, "id": 29937550, "properties": {}, "type": "Feature"}, {"geometry": {"coordinates": [-122.3134473, 47.6536313], "type": "Point"}, "id": 29937555, "properties": {"kerb": "lowered"}, "type": "Feature"}, {"geometry": {"coordinates": [-122.3132415, 47.6535512], "type": "Point"}, "id": 29937556, "properties": {}, "type": "Feature"}, {"geometry": {"coordinates": [-122.3123247, 47.6531178], "type": "Point"}, "id": 29937559, "properties": {"bicycle": "yes", "crossing": "traffic_signals", "highway": "crossing"}, "type": "Feature

In [51]:
OSM = gpd.GeoDataFrame.from_features(response)

In [53]:
OSM[OSM.geometry.type == 'LineString']

,access,addr:city,addr:housenumber,addr:postcode,addr:street,barrier,bench,bicycle,bus,created_by,...,railway,ref,shelter,source,supervised,surface,swing_gate:type,tactile_paving,traffic_calming,wheelchair
